# Retail Online Retail II

In [2]:

import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report


In [3]:
df = pd.read_csv("online_retail_II.csv")

# Understanding the Dataset

In [4]:
print("\n=== Shape ===")
print(df.shape)

print("\n=== Columns ===")
print(df.columns)

print("\n=== Data Types ===")
print(df.dtypes)

print("\n=== First Few Rows ===")
print(df.head())


=== Shape ===
(1067371, 8)

=== Columns ===
Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

=== Data Types ===
Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

=== First Few Rows ===
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  
0  2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1  2009-12-01 07:45:00   6.75   

## Cleaning

In [5]:
# Remove rows without Customer ID (can't use them for any customer modeling)
df = df.dropna(subset=["Customer ID"])


In [6]:
# Remove negative quantities (returns) and zero/negative price rows
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)]

In [7]:
# Convert Customer ID to integer (currently float)
df["Customer ID"] = df["Customer ID"].astype(int)

In [8]:
# Create Revenue column (Quantity × Price)
df["Revenue"] = df["Quantity"] * df["Price"]

print("Cleaned shape:", df.shape)
print(df.head())

Cleaned shape: (805549, 9)
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  Revenue  
0  2009-12-01 07:45:00   6.95        13085  United Kingdom     83.4  
1  2009-12-01 07:45:00   6.75        13085  United Kingdom     81.0  
2  2009-12-01 07:45:00   6.75        13085  United Kingdom     81.0  
3  2009-12-01 07:45:00   2.10        13085  United Kingdom    100.8  
4  2009-12-01 07:45:00   1.25        13085  United Kingdom     30.0  


In [9]:
# Create a copy so   
clean_df = df.copy()


In [10]:
# Basic cleaning
clean_df = clean_df.dropna(subset=["Customer ID"])
clean_df = clean_df[(clean_df["Quantity"] > 0) & (clean_df["Price"] > 0)]
clean_df["Customer ID"] = clean_df["Customer ID"].astype(int)
clean_df["Revenue"] = clean_df["Quantity"] * clean_df["Price"]

print("Raw shape:", df.shape)
print("Cleaned shape:", clean_df.shape)
print(clean_df.head())

Raw shape: (805549, 9)
Cleaned shape: (805549, 9)
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  Revenue  
0  2009-12-01 07:45:00   6.95        13085  United Kingdom     83.4  
1  2009-12-01 07:45:00   6.75        13085  United Kingdom     81.0  
2  2009-12-01 07:45:00   6.75        13085  United Kingdom     81.0  
3  2009-12-01 07:45:00   2.10        13085  United Kingdom    100.8  
4  2009-12-01 07:45:00   1.25        13085  United Kingdom     30.0  


# RFM (Recency, Frequency & Monetary) Feature Engineering 

In [11]:
# ensure InvoiceDate is datetime
clean_df["InvoiceDate"] = pd.to_datetime(clean_df["InvoiceDate"])

# reference date = last date in dataset
reference_date = clean_df["InvoiceDate"].max()

# group by Customer ID
rfm = clean_df.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (reference_date - x.max()).days,  # Recency
    "Invoice": "nunique",                                      # Frequency
    "Revenue": "sum"                                           # Monetary
})

# rename columns
rfm.columns = ["Recency", "Frequency", "Monetary"]

print("\n=== RFM Table Preview ===")
print(rfm.head())
print("\nRFM shape:", rfm.shape)



=== RFM Table Preview ===
             Recency  Frequency  Monetary
Customer ID                              
12346            325         12  77556.46
12347              1          8   5633.32
12348             74          5   2019.40
12349             18          4   4428.69
12350            309          1    334.40

RFM shape: (5878, 3)


In [12]:
rfm.to_csv("rfm.csv")


In [13]:
rfm = pd.read_csv("rfm.csv", index_col="Customer ID")

In [14]:


# ----------------------------------------
# 1. Scale the RFM features
# ----------------------------------------
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm)



In [15]:
# ----------------------------------------
# 2. Run K-Means clustering
# Choose 4 clusters (standard starting point)
# ----------------------------------------
kmeans = KMeans(n_clusters=4, random_state=42)
rfm["Segment"] = kmeans.fit_predict(rfm_scaled)



In [16]:
# ----------------------------------------
# 3. Print preview
# ----------------------------------------
print(rfm.head())
print("\nSegment counts:")
print(rfm["Segment"].value_counts())




             Recency  Frequency  Monetary  Segment
Customer ID                                       
12346            325         12  77556.46        1
12347              1          8   5633.32        1
12348             74          5   2019.40        1
12349             18          4   4428.69        1
12350            309          1    334.40        0

Segment counts:
Segment
1    3841
0    1998
2      35
3       4
Name: count, dtype: int64


In [17]:
# ----------------------------------------
# Save the segmented RFM table
# ----------------------------------------
rfm.to_csv("rfm_segmented.csv")
print("\nSaved segmentation to rfm_segmented.csv")


Saved segmentation to rfm_segmented.csv


In [18]:
print(rfm.groupby("Segment").mean())
print(rfm.groupby("Segment").size())


            Recency   Frequency       Monetary
Segment                                       
0        462.032032    2.212212     765.244446
1         66.005728    7.307732    3009.402451
2         24.942857  103.714286   83086.079771
3          2.500000  212.500000  436835.792500
Segment
0    1998
1    3841
2      35
3       4
dtype: int64


| Segment | Name                  |
| ------- | --------------------- |
| 3       | **Super VIPs**        |
| 1       | **VIP Loyalists**     |
| 2       | **Regular Customers** |
| 0       | **At-Risk Customers** |


🟩 Segment 3 — “Elite Super-VIPs”

Recency: 2.5 days → bought extremely recently

Frequency: 212 → insane repeat purchases

Monetary: 436,000+ → unbelievably high revenue

Count: 4 customers

📌 These are absolute outliers.
They are 0.06% of customers but maybe 10–15% of revenue.

👉 Label:
“Super VIP / Ultra High-Value Customers”

🟦 Segment 1 — “VIP Loyalists”

Recency: 25 days → recently active

Frequency: 104 → extremely frequent

Monetary: 83,000 → huge spend

Count: 35 customers

These are your top-tier core loyal customers (big buyers, active, and consistent).

👉 Label:
“VIP Loyalists” / “High-Value Active Buyers”

🟨 Segment 2 — “Promising / Regular Buyers”

Recency: 66 days

Frequency: 7.3

Monetary: 3,009

Count: 3841 (largest group)

These look like your normal, healthy, mid-value customers.

👉 Label:
“Regular Buyers / Mid-Value Customers”

🟥 Segment 0 — “Lapsed / At-Risk”

Recency: 462 days → haven’t bought in over a year

Frequency: 2 → rarely bought

Monetary: 765

Count: 1998

These are almost dead customers.
They either churned or were never valuable.

👉 Label:
“At-Risk / Inactive Customers”

In [19]:
segment_labels = {
    3: "Super VIPs",
    1: "VIP Loyalists",
    2: "Regular Customers",
    0: "At-Risk Customers"
}

rfm["SegmentName"] = rfm["Segment"].map(segment_labels)
print(rfm.head())
rfm.to_csv("rfm_segmented_with_names.csv")


             Recency  Frequency  Monetary  Segment        SegmentName
Customer ID                                                          
12346            325         12  77556.46        1      VIP Loyalists
12347              1          8   5633.32        1      VIP Loyalists
12348             74          5   2019.40        1      VIP Loyalists
12349             18          4   4428.69        1      VIP Loyalists
12350            309          1    334.40        0  At-Risk Customers


# CLV Prediction (Future Spend Model)

In [20]:
# Load the cleaned data you created earlier
clean_df



,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680,France,14.85


In [21]:
# Keep only what we need for CLV
clv_df = clean_df[["Customer ID", "InvoiceDate","Invoice", "Revenue"]].copy()

print(clv_df.head())
print(clv_df.shape)

# Save for next steps
clv_df.to_csv("clv_base.csv", index=False)
print("Saved base CLV dataset: clv_base.csv")


   Customer ID         InvoiceDate Invoice  Revenue
0        13085 2009-12-01 07:45:00  489434     83.4
1        13085 2009-12-01 07:45:00  489434     81.0
2        13085 2009-12-01 07:45:00  489434     81.0
3        13085 2009-12-01 07:45:00  489434    100.8
4        13085 2009-12-01 07:45:00  489434     30.0
(805549, 4)
Saved base CLV dataset: clv_base.csv


In [22]:
# 1. Check the date range
print("Min date:", clv_df["InvoiceDate"].min())
print("Max date:", clv_df["InvoiceDate"].max())




Min date: 2009-12-01 07:45:00
Max date: 2011-12-09 12:50:00


In [23]:
print(clv_df.columns)
# should show: ['Customer ID', 'InvoiceDate', 'Invoice', 'Revenue']


Index(['Customer ID', 'InvoiceDate', 'Invoice', 'Revenue'], dtype='object')


In [24]:

# 2. Choose cutoff date (last 3 months)
max_date = clv_df["InvoiceDate"].max()
cutoff_date = max_date - pd.Timedelta(days=90)

print("\nCutoff date:", cutoff_date)





Cutoff date: 2011-09-10 12:50:00


In [25]:
# 3. Split into past (features window) and future (target window)
past_df = clv_df[clv_df["InvoiceDate"] <= cutoff_date]
future_df = clv_df[
    (clv_df["InvoiceDate"] > cutoff_date) &
    (clv_df["InvoiceDate"] <= cutoff_date + pd.Timedelta(days=90))
]

print("Past rows:", past_df.shape)
print("Future rows:", future_df.shape)

Past rows: (644022, 4)
Future rows: (161527, 4)


In [26]:
# 4. Build FEATURES from past window (as of cutoff_date)
clv_features = past_df.groupby("Customer ID").agg(
    Recency=("InvoiceDate", lambda x: (cutoff_date - x.max()).days),
    Frequency=("InvoiceDate", "count"),     # number of past transactions (rows)
    Monetary=("Revenue", "sum")             # total past spend
)

print("\nCLV features preview:")
print(clv_features.head())



CLV features preview:
             Recency  Frequency  Monetary
Customer ID                              
12346            235         34  77556.46
12347             39        195   4114.18
12348            158         48   1709.40
12349            317        102   2671.14
12350            219         17    334.40


In [27]:
# 5. Build TARGET from future window: total spend in next 3 months
future_spend = future_df.groupby("Customer ID")["Revenue"].sum()
future_spend = future_spend.rename("future_spend_3m")

print("\nFuture spend (target) preview:")
print(future_spend.head())




Future spend (target) preview:
Customer ID
12347    1519.14
12348     310.00
12349    1757.55
12352     944.23
12356      58.35
Name: future_spend_3m, dtype: float64


In [28]:
# 6. Merge features + target
clv_training = clv_features.join(future_spend, how="left")
clv_training["future_spend_3m"] = clv_training["future_spend_3m"].fillna(0)

print("\nCLV training data preview:")
print(clv_training.head())
print("\nShape:", clv_training.shape)




CLV training data preview:
             Recency  Frequency  Monetary  future_spend_3m
Customer ID                                               
12346            235         34  77556.46             0.00
12347             39        195   4114.18          1519.14
12348            158         48   1709.40           310.00
12349            317        102   2671.14          1757.55
12350            219         17    334.40             0.00

Shape: (5281, 4)


In [29]:
# 7. Save to CSV for modeling
clv_training.to_csv("clv_training_data.csv")
print("\nSaved CLV training dataset to clv_training_data.csv")


Saved CLV training dataset to clv_training_data.csv


Frequency here = number of rows per customer in the past window, which is close to “number of items/lines” rather than number of invoices. It’s fine for now; if later you want true invoice frequency, we’ll need to include Invoice in clv_df as well.

# CLV Model

In [30]:
# 1. Load training dataset
df = pd.read_csv("clv_training_data.csv")

In [31]:
# 2. Define features and target
X = df[["Recency", "Frequency", "Monetary"]]
y = df["future_spend_3m"]


In [32]:
# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [33]:
# 4. Model: RandomForest Regressor (simple, no tuning)
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    random_state=42
)

model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [34]:
# 5. Predictions
y_pred = model.predict(X_test)


In [35]:
# 6. Evaluation
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("\n=== CLV Model Performance ===")
print("MAE:", mae)
print("RMSE:", rmse)



=== CLV Model Performance ===
MAE: 604.8114048864711
RMSE: 5617.436517579893


In [36]:
# 7. Feature importance
importances = model.feature_importances_
for name, value in zip(X.columns, importances):
    print(f"{name}: {value:.4f}")


Recency: 0.0512
Frequency: 0.1167
Monetary: 0.8321


# log-transformed target

In [37]:
df = pd.read_csv("clv_training_data.csv")

# Add log-transformed target
df["future_spend_3m_log"] = np.log1p(df["future_spend_3m"])

df.to_csv("clv_training_data_log.csv", index=False)
print("Saved clv_training_data_log.csv")
print(df[["future_spend_3m", "future_spend_3m_log"]].head())


Saved clv_training_data_log.csv
   future_spend_3m  future_spend_3m_log
0             0.00             0.000000
1          1519.14             7.326558
2           310.00             5.739793
3          1757.55             7.472245
4             0.00             0.000000


In [38]:
# Load dataset with log target
df = pd.read_csv("clv_training_data_log.csv")

# Features
X = df[["Recency", "Frequency", "Monetary"]]
y = df["future_spend_3m_log"]   # <-- log version


In [39]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Random Forest
model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)
model.fit(X_train, y_train)


RandomForestRegressor(n_estimators=300, random_state=42)

In [40]:
# Predict log target
y_pred_log = model.predict(X_test)

In [41]:
# Convert back to money scale
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# Metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print("MAE:", mae)
print("RMSE:", rmse)

MAE: 568.1951198319751
RMSE: 5625.029558907921


The log-transform alone didn’t fix the underlying issue because the model (RandomForest) is not good at handling long-tailed regression.

# LightGBM CLV Model

LightGBM handles:

skewed numeric targets

long-tailed spending

irregular behaviors

nonlinear interactions

imbalanced data

In [42]:
# 1. Load dataset with log target
df = pd.read_csv("clv_training_data_log.csv")

In [43]:
# 2. Define features and target
X = df[["Recency", "Frequency", "Monetary"]]
y = df["future_spend_3m_log"]


In [44]:
# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [45]:
# 4. LightGBM model (simple)
model = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    random_state=42
)

model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000499 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 4224, number of used features: 3
[LightGBM] [Info] Start training from score 2.732957


LGBMRegressor(learning_rate=0.05, n_estimators=500, random_state=42)

In [46]:
# 5. Predict log target
y_pred_log = model.predict(X_test)

In [47]:
# 6. Convert back to money scale
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# 7. Metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print("\n=== LightGBM CLV Model Performance ===")
print("MAE:", mae)
print("RMSE:", rmse)


=== LightGBM CLV Model Performance ===
MAE: 616.877804346771
RMSE: 5702.1422537192


In [48]:
# 8. Feature importance
print("\nFeature Importances:")
for name, importance in zip(X.columns, model.feature_importances_):
    print(f"{name}: {importance}")



Feature Importances:
Recency: 4865
Frequency: 5160
Monetary: 4975


# FIXING CLV

Even with log transform, the future spend has too many:

zeros

few massive outliers (several customers spend tens of thousands)

This destroys RMSE.




Frequency=("InvoiceDate","count")

This counts ROWS, not INVOICES.

Meaning:

Someone buying 10 items in 1 invoice = frequency 10

Someone buying 10 times in 10 separate orders = frequency 10

These are not the same behavior.

It weakens predictive power.

We must use Invoice column for correct frequency.

In [49]:
clv_features = past_df.groupby("Customer ID").agg(
    Recency=("InvoiceDate", lambda x: (cutoff_date - x.max()).days),
    Frequency=("Invoice", "nunique"),   # 🔹 number of unique invoices
    Monetary=("Revenue", "sum")
)


In [50]:
future_spend = future_df.groupby("Customer ID")["Revenue"].sum()
future_spend = future_spend.rename("future_spend_3m")

clv_training = clv_features.join(future_spend, how="left")
clv_training["future_spend_3m"] = clv_training["future_spend_3m"].fillna(0)

clv_training.to_csv("clv_training_data.csv")


Re-Running LightG

In [51]:
# 1. Load CLV training data
df = pd.read_csv("clv_training_data.csv")
print("Columns:", df.columns.tolist())
print("Data shape:", df.shape)


Columns: ['Customer ID', 'Recency', 'Frequency', 'Monetary', 'future_spend_3m']
Data shape: (5281, 5)


In [52]:
# 2. Clip extreme outliers in future spend (top 0.5%)
upper = df["future_spend_3m"].quantile(0.995)
df["future_spend_3m_clipped"] = df["future_spend_3m"].clip(upper=upper)

In [53]:
# 3. Log-transform target
df["future_spend_3m_log"] = np.log1p(df["future_spend_3m_clipped"])


In [54]:
# 4. Features and target
X = df[["Recency", "Frequency", "Monetary"]]
y = df["future_spend_3m_log"]

In [55]:
# 5. Train–test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [56]:
# 6. LightGBM model (simple, but strong)
model = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    random_state=42
)

model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000305 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 562
[LightGBM] [Info] Number of data points in the train set: 4224, number of used features: 3
[LightGBM] [Info] Start training from score 2.729319


LGBMRegressor(learning_rate=0.05, n_estimators=500, random_state=42)

In [57]:
# 7. Predict (log space → money space)
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)   # back to money scale
y_true = np.expm1(y_test)


In [58]:
# 8. Metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print("\n=== LightGBM CLV Model Performance ===")
print("MAE:", mae)
print("RMSE:", rmse)



=== LightGBM CLV Model Performance ===
MAE: 361.86084286809285
RMSE: 986.1172050060918


In [59]:
# 9. Feature importance
print("\nFeature Importances:")
for name, importance in zip(X.columns, model.feature_importances_):
    print(f"{name}: {importance}")


Feature Importances:
Recency: 6451
Frequency: 1988
Monetary: 6561


In [60]:
# 10. (Optional) Save model predictions for later use
results = X_test.copy()
results["true_future_spend_3m"] = y_true
results["pred_future_spend_3m"] = y_pred
results.to_csv("clv_predictions_sample.csv", index=False)
print("\nSaved sample predictions to clv_predictions_sample.csv")


Saved sample predictions to clv_predictions_sample.csv


In [61]:
model.booster_.save_model("clv_model.txt")


# SCORE ALL CUSTOMERS WITH PREDICTED CLV

In [62]:
# 1. Load CLV training data
df = pd.read_csv("clv_training_data.csv")

In [63]:
# 2. Clip extreme values like during training
upper = df["future_spend_3m"].quantile(0.995)
df["future_spend_3m_clipped"] = df["future_spend_3m"].clip(upper=upper)

In [64]:
# 3. Create log target (same as training)
df["future_spend_3m_log"] = np.log1p(df["future_spend_3m_clipped"])

In [65]:
# 4. Load the LightGBM model (saved model
model = lgb.Booster(model_file="clv_model.txt")


In [66]:
# 5. Features
X = df[["Recency", "Frequency", "Monetary"]]# 5. Features
X = df[["Recency", "Frequency", "Monetary"]]

In [67]:
# 6. Predict log target
pred_log = model.predict(X)

In [68]:
# 7. Convert back to money scale
df["predicted_clv"] = np.expm1(pred_log)

In [69]:
# 8. Save scored customers
df[["Customer ID", "Recency", "Frequency", "Monetary", 
    "future_spend_3m", "predicted_clv"]].to_csv(
    "clv_scored_customers.csv", index=False
)

print("Saved clv_scored_customers.csv")

Saved clv_scored_customers.csv


# Targeting Engine (CLV-based)

In [70]:
# 1. Load scored customers
df = pd.read_csv("clv_scored_customers.csv")

# Safety check
print("Loaded rows:", df.shape[0])
print(df.head())

Loaded rows: 5281
   Customer ID  Recency  Frequency  Monetary  future_spend_3m  predicted_clv
0        12346      235         12  77556.46             0.00       2.123474
1        12347       39          6   4114.18          1519.14     131.739108
2        12348      158          4   1709.40           310.00      74.371905
3        12349      317          3   2671.14          1757.55     738.822092
4        12350      219          1    334.40             0.00      -0.218987


In [71]:
# 2. Set campaign parameters
# CHANGE these numbers depending on your scenario
total_budget = 10000.0          # total marketing budget in currency units
cost_per_contact = 2.0          # how much it costs to contact one customer (email/SMS etc.)

In [72]:
# 3. How many customers can we afford to contact?
max_customers = int(total_budget // cost_per_contact)
max_customers = min(max_customers, df.shape[0])   # can't exceed number of customers

print(f"\nTotal budget: {total_budget}")
print(f"Cost per contact: {cost_per_contact}")
print(f"Max customers we can contact: {max_customers}")


Total budget: 10000.0
Cost per contact: 2.0
Max customers we can contact: 5000


In [73]:
# 4. Sort customers by predicted CLV (highest first)
df_sorted = df.sort_values("predicted_clv", ascending=False)

In [74]:
# 5. Select top N customers
target_df = df_sorted.head(max_customers).copy()

In [75]:
# 6. Calculate expected revenue and simple ROI
expected_revenue = target_df["predicted_clv"].sum()
expected_cost = max_customers * cost_per_contact
expected_roi = expected_revenue / expected_cost if expected_cost > 0 else np.nan

print("\n=== Campaign Summary ===")
print("Expected revenue from targeted customers:", expected_revenue)
print("Expected campaign cost:", expected_cost)
print("Expected ROI (revenue / cost):", expected_roi)


=== Campaign Summary ===
Expected revenue from targeted customers: 1076733.7330326743
Expected campaign cost: 10000.0
Expected ROI (revenue / cost): 107.67337330326743


In [76]:
# 7. Save target list
cols_to_save = ["Customer ID", "Recency", "Frequency", "Monetary", "predicted_clv"]
if "future_spend_3m" in target_df.columns:
    cols_to_save.append("future_spend_3m")  # keep true future spend if available

target_df[cols_to_save].to_csv("campaign_target_list.csv", index=False)
print("\nSaved campaign target list to 'campaign_target_list.csv'")



Saved campaign target list to 'campaign_target_list.csv'


## Campaign Targeting Engine (Explanation Summary)

After generating predicted CLV values for every customer in the dataset, the next step is to convert these predictions into actionable marketing decisions. The goal of this module is to design an AI-driven targeting engine that selects the best customers to target under a given marketing budget.

## 1. Business Problem

Marketing campaigns have limited budgets.
Every contact (email, SMS, discount, outreach) costs money.
Therefore:

Which customers should we target to maximize expected revenue?

Instead of selecting customers randomly or based on intuition, we use the predicted CLV from our ML model to make data-driven decisions.

## 2. Core Logic of the Targeting Engine

The engine uses three main inputs:

Predicted CLV: expected future value of each customer

Total campaign budget

Cost per contact (how much it costs to reach 1 customer)

From this, we compute:

max_customers = floor(budget / cost_per_contact)


This tells us how many customers we can afford to target.

Then we:

Sort customers by predicted CLV (highest to lowest)

Select the top N customers that fit within the budget

Estimate expected revenue = sum of predicted CLVs of selected customers

Compute ROI = expected revenue / campaign cost

This creates a direct link between:

Marketing spend → Customer selection → Expected financial return

## 3. Why This Works

This method ensures the marketing campaign focuses on customers who are most likely to generate value in the near future. Instead of spending money equally or randomly:

We prioritize high-value customers

We avoid wasting money on low-spending or churned customers

ROI is predictable before running the campaign

Finance, marketing, and analytics teams can align on budget allocation

This is the foundation of modern AI-driven CRM and growth marketing.

## 4. Output

The engine generates a file:

campaign_target_list.csv


containing:

Customer ID

Behavioral features (Recency, Frequency, Monetary)

Predicted CLV

(Optional) True future spend (for evaluation)

This file becomes the final targeting list used by marketing or downstream automation systems.

## 5. What This Enables Next

The CLV-based targeting engine (B) forms the first half of the retail AI system.
The next component (C) is Propensity Modeling — predicting who is likely to buy.

Later, we combine:

Propensity scores (likelihood of purchase)

CLV predictions (expected value)

to build a full growth engine that prioritizes high-value AND high-probability customers.

## Summary Sentence

Part B converts machine learning predictions into a real business decision pipeline: selecting the optimal customers to target under a budget, maximizing expected revenue using CLV-driven ranking.

## Part C: Purchase Propensity Model (Classification)

1 = customer bought at least once in the future window (next 3 months)
0 = customer bought nothing

In [77]:
# Build binary propensity label
future_purchase_flag = (future_spend > 0).astype(int)
future_purchase_flag = future_purchase_flag.rename("purchase_next_3m")

In [78]:
propensity_data = clv_features.join(future_purchase_flag, how="left")
propensity_data["purchase_next_3m"] = propensity_data["purchase_next_3m"].fillna(0)

In [79]:
propensity_data.to_csv("propensity_training_data.csv")
print("Saved propensity_training_data.csv")

Saved propensity_training_data.csv


## C2 — Train Propensity Model (LightGBM Classifier)

In [80]:
# 1. Load data
df = pd.read_csv("propensity_training_data.csv")

In [81]:
# 2. Features + Target
X = df[["Recency", "Frequency", "Monetary"]]
y = df["purchase_next_3m"]


In [82]:
# 3. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [83]:
# 4. LightGBM classifier
model = lgb.LGBMClassifier(
    n_estimators=300,
    learning_rate=0.05,
    class_weight="balanced",  # handles the imbalance
    random_state=42
)

model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 1833, number of negative: 2391
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000617 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 561
[LightGBM] [Info] Number of data points in the train set: 4224, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


LGBMClassifier(class_weight='balanced', learning_rate=0.05, n_estimators=300,
               random_state=42)

In [84]:
# 5. Predict probabilities
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)

In [85]:
# 6. Evaluate
acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print("\n=== Propensity Model Performance ===")
print("Accuracy:", acc)
print("AUC:", auc)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


=== Propensity Model Performance ===
Accuracy: 0.7000946073793756
AUC: 0.775005282677917

Classification Report:
               precision    recall  f1-score   support

         0.0       0.74      0.73      0.73       598
         1.0       0.65      0.67      0.66       459

    accuracy                           0.70      1057
   macro avg       0.70      0.70      0.70      1057
weighted avg       0.70      0.70      0.70      1057



In [86]:
# 7. Feature importance
print("\nFeature Importances:")
for name, value in zip(X.columns, model.feature_importances_):
    print(f"{name}: {value}")

# 8. Save model
model.booster_.save_model("propensity_model.txt")
print("\nSaved model: propensity_model.txt")



Feature Importances:
Recency: 3745
Frequency: 1200
Monetary: 4055

Saved model: propensity_model.txt


## Score ALL customers with purchase probability

In [87]:
# Load data & model
df = pd.read_csv("propensity_training_data.csv")
model = lgb.Booster(model_file="propensity_model.txt")

In [88]:
# Features
X = df[["Recency", "Frequency", "Monetary"]]

In [89]:
# Predict probability of future purchase
df["purchase_probability"] = model.predict(X)

df.to_csv("propensity_scored_customers.csv", index=False)
print("Saved: propensity_scored_customers.csv")

Saved: propensity_scored_customers.csv


## Part C — Propensity Modeling (Summary)

To predict which customers are likely to make a purchase in the next 3 months, I trained a LightGBM classification model using only RFM features (Recency, Frequency, Monetary). The target variable was a binary flag indicating whether the customer made at least one future purchase in the prediction window.

The model achieved strong performance with AUC = 0.775 and balanced precision/recall across both classes, indicating it can effectively rank customers by purchase likelihood. Recency and Monetary emerged as the strongest predictors, consistent with purchasing behavior in retail datasets.

This propensity model will later be combined with the CLV model to create a unified targeting engine that selects customers who are both likely to buy and expected to generate high value, forming the core of the Retail AI Growth Engine.

In [91]:
clean_df.to_csv("clean_df.csv", index=False)


KeyboardInterrupt: 

In [93]:
clean_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0
